# 🚀 **JLSatiro Clipper AI - V23.0 Titanium Edition**

Bem-vindo ao **JLSatiro Clipper AI**, a plataforma definitiva para criação de cortes virais automatizados.

### ⚡ **Recursos Titanium:**
- **Smart Crop V2.5**: Rastreamento facial ultra-preciso com MediaPipe (Fixed).
- **Hyper-Speed Transcription**: Transcrição GPU acelerada com Faster-Whisper.
- **Auto-Viral**: Geração automática de Thumbnails e Hooks.
- **100% Local**: Otimizado para rodar localmente no Colab com zero delay.
- **Titanium Vault V2**: Persistência via Arquivo Único (Turbo Load). Setup de 5 minutos -> 20 segundos.

---

In [ ]:
#@title 1. 🛠️ Instalação do Sistema (Titanium Vault - Turbo Mode)
#@markdown O sistema carrega um pacote único do Drive para a RAM/SSD. Velocidade Máxima.

import os
import subprocess
import sys
import shutil
import time

print("💎 [JLSatiro] Iniciando Setup Titanium V23.0...")

# --- DRIVE MOUNT & CONFIG ---
print("☁️ [0/5] Conectando Drive...\n")
try:
    from google.colab import drive
    drive.mount('/content/drive')
    
    CONFIG_DIR = "/content/drive/MyDrive/JLSatiro_Config"
    VAULT_FILE = os.path.join(CONFIG_DIR, "titanium_libs_v24_stable.tar.gz") # Force Fresh Install (V24)
    LOCAL_LIB_PATH = "/content/titanium_libs"
    
    os.makedirs(CONFIG_DIR, exist_ok=True)
    print(f"   ✅ Drive Conectado! Pacote Turbo: {VAULT_FILE}")
except Exception as e:
    print(f"   ⚠️ Aviso: Falha no Drive ({e}). Modo volátil (RAM) ativado.")
    CONFIG_DIR = None
    VAULT_FILE = None
    LOCAL_LIB_PATH = "/content/titanium_libs"

# Configurar Path Python
if LOCAL_LIB_PATH not in sys.path:
    sys.path.insert(0, LOCAL_LIB_PATH)
os.environ["PYTHONPATH"] = f"{LOCAL_LIB_PATH}:{os.environ.get('PYTHONPATH', '')}"

# --- VAULT RESTORE ---
vault_restored = False

if VAULT_FILE and os.path.exists(VAULT_FILE) and not os.path.exists(".setup_done"):
    print(f"\n⚡ [VAULT] Backup Turbo encontrado! Restaurando... (Isso é muito rápido)")
    try:
        # 1. Copy (Single File = Fast)
        print("   📂 Baixando Pacote do Drive...")
        subprocess.run(f"cp '{VAULT_FILE}' /content/packet.tar.gz", shell=True)
        
        # 2. Extract (SSD = Fast)
        print("   📦 Descompactando...")
        os.makedirs(LOCAL_LIB_PATH, exist_ok=True)
        subprocess.run(f"tar -xzf /content/packet.tar.gz -C {LOCAL_LIB_PATH}", shell=True)
        os.remove("/content/packet.tar.gz") # Cleanup
        
        # 3. System Deps (Fast)
        print("   🔧 Restaurando Drivers...")
        subprocess.run("apt-get update -qq && apt-get install -y ffmpeg imagemagick libsndfile1 sox > /dev/null", shell=True)
        
        # Fix Magick & HF
        if os.path.exists("/etc/ImageMagick-6/policy.xml"):
             subprocess.run("sed -i 's/none/read,write/g' /etc/ImageMagick-6/policy.xml", shell=True)

        # FORCE HF HOTFIX ON EVERY RESTORE TO BE SAFE
        subprocess.run(f"{sys.executable} -m pip install 'huggingface_hub<0.25.0' --target {LOCAL_LIB_PATH} --no-deps", shell=True, stdout=subprocess.DEVNULL)

        vault_restored = True
        with open(".setup_done", "w") as f: f.write("restored")
        print("   ✅ SISTEMA PRONTO EM SEGUNDOS!")
    except Exception as e:
        print(f"   ❌ Falha na Restauração: {e}. Reinstalando...")

# --- FRESH INSTALL & BACKUP ---
if not vault_restored and not os.path.exists(".setup_done"):
    # 1. GPU Check
    print("\n🖥️ [1/4] Verificando Hardware...")
    try: subprocess.run(["nvidia-smi"], check=True)
    except: print("   ⚠️ GPU não detectada.")

    # 2. Apt
    print("📦 [2/4] Instalando Motores (FFmpeg)...")
    subprocess.run("apt-get update -qq && apt-get install -y ffmpeg imagemagick libsndfile1 sox -qq", shell=True)
    
    if os.path.exists("/etc/ImageMagick-6/policy.xml"):
        subprocess.run("sed -i 's/none/read,write/g' /etc/ImageMagick-6/policy.xml", shell=True)

    # 3. Pip
    print(f"🐍 [3/4] Construindo Ambiente Python em {LOCAL_LIB_PATH}...")
    os.makedirs(LOCAL_LIB_PATH, exist_ok=True)

    # GPU LLM
    subprocess.run(f'pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121 --no-cache-dir --target {LOCAL_LIB_PATH}', shell=True)

    libraries = [
        "torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121",
        "faster-whisper>=1.0.0",
        "openai-whisper",
        "mediapipe>=0.10.14",
        "transformers",
        "instructor>=0.5.0,<1.0.0",
        "pydantic",
        "moviepy",
        "opencv-python-headless",
        "librosa",
        "soundfile",
        "imageio",
        "imageio-ffmpeg",
        "pillow",
        "pydub",
        "gradio>=4.0.0,<5.0.0",
        "flask",
        "flask-cors",
        "pytubefix>=6.0.0",
        "yt-dlp",
        "python-dotenv",
        "numpy<2.0.0",
        "pandas",
        "edge-tts",
        "huggingface_hub<0.25.0"
    ]

    pip_cmd = f"{sys.executable} -m pip install --upgrade --no-cache-dir --target {LOCAL_LIB_PATH}"
    for lib in libraries:
        subprocess.run(f"{pip_cmd} \"{lib}\"", shell=True, stdout=subprocess.DEVNULL)

    # 4. CREATE BACKUP (One-Time Slow)
    if VAULT_FILE:
        print("💾 [VAULT] Compactando e Enviando Backup Único para o Drive... (Aguarde)")
        try:
             subprocess.run(f"cd {LOCAL_LIB_PATH} && tar -czf /content/packet.tar.gz .", shell=True)
             shutil.copy("/content/packet.tar.gz", VAULT_FILE)
             print("   ✅ BACKUP TURBO SALVO!")
        except Exception as e: print(f"   ⚠️ Erro backup: {e}")

    with open(".setup_done", "w") as f: f.write("ok")

elif os.path.exists(".setup_done"):
    print("\n⚡ [CACHE] Sistema já ativo!")


# 5. CONFIG WORKSPACE
print("📂 [5/5] Sincronizando Código...")
repo_url = "https://github.com/noviso123/JLSatiroClipperAI.git"
repo_dir = "/content/JLSatiroClipperAI"

if not os.path.exists(repo_dir):
    subprocess.run(f"git clone {repo_url} {repo_dir}", shell=True)
else:
    subprocess.run(f"cd {repo_dir} && git pull", shell=True)

# RESTORE CREDENTIALS
if CONFIG_DIR:
    files_to_sync = ["client_secret.json", "token.pickle", "cookies.txt"]
    for f_name in files_to_sync:
        src = os.path.join(CONFIG_DIR, f_name)
        dst = os.path.join(repo_dir, f_name)
        if os.path.exists(src):
             if not os.path.exists(dst) or (os.path.getmtime(src) > os.path.getmtime(dst)):
                 shutil.copy(src, dst)
                 print(f"   ♻️ Login Restaurado: {f_name}")

# Criar pastas necessárias
os.makedirs(f"{repo_dir}/downloads", exist_ok=True)
os.makedirs(f"{repo_dir}/temp_work", exist_ok=True)
os.makedirs(f"{repo_dir}/backend/models", exist_ok=True)

print("\n✨ SETUP PRONTO! Inicie abaixo.")

In [ ]:
#@title 2. 🚀 Iniciar JLSatiro Clipper AI
#@markdown Clique no play e aguarde. O app irá usar as bibliotecas carregadas do Vault.

import os
import sys
import shutil

# Configurar paths novamente para garantir
LOCAL_LIB_PATH = "/content/titanium_libs"
if os.path.exists(LOCAL_LIB_PATH) and LOCAL_LIB_PATH not in sys.path:
    sys.path.insert(0, LOCAL_LIB_PATH)

# Garantir que estamos no diretório correto
os.chdir("/content/JLSatiroClipperAI")

# Forçar patch do MediaPipe
video_engine_path = "backend/video_engine.py"
if os.path.exists(video_engine_path):
    with open(video_engine_path, 'r', encoding='utf-8') as f: content = f.read()
    if "from mediapipe import solutions" in content:
        content = content.replace("from mediapipe import solutions\n\n        mp_face_detection = solutions.face_detection", "import mediapipe as mp\n\n        mp_face_detection = mp.solutions.face_detection")
        content = content.replace("from mediapipe import solutions\n\n        mp_face = solutions.face_detection", "import mediapipe as mp\n\n        mp_face = mp.solutions.face_detection")
        with open(video_engine_path, 'w', encoding='utf-8') as f: f.write(content)

try:
    !python app.py
except KeyboardInterrupt:
    pass
finally:
    # BACKUP PARA O DRIVE AO FECHAR (Apenas configs leves)
    CONFIG_DIR = "/content/drive/MyDrive/JLSatiro_Config"
    if os.path.exists(CONFIG_DIR):
        print("\n💾 [AUTO-BACKUP] Salvando logins...")
        files_to_sync = ["client_secret.json", "token.pickle", "cookies.txt"]
        for f_name in files_to_sync:
            if os.path.exists(f_name):
                 dst = os.path.join(CONFIG_DIR, f_name)
                 if not os.path.exists(dst) or (os.path.getmtime(f_name) > os.path.getmtime(dst)):
                    shutil.copy(f_name, dst)
                    print(f"   ✅ Salvo: {f_name}")